In [1]:
# Importing dependencies 
import pandas as pd
import numpy as np
import scipy.stats as st
import os

# Importing  both time traveled to work csvs and concatonating into a sigle dataframe
time_traveled_dir = ['Travel Time to Work', 'Travel Time County and State Level']
for dir in time_traveled_dir:

    path = f'Resources/{dir}'
    files = os.listdir(path)

    travel_df = pd.DataFrame()
    cs_travel_df = pd.DataFrame()

    for file in files:
        if dir == 'Travel Time to Work':
            temp = pd.read_csv(path + '\\' + file, skiprows=[1])
            temp = temp[['B08303_001E', 'B08303_002E', 'B08303_003E', 'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E', \
                'B08303_008E', 'B08303_009E', 'B08303_010E', 'B08303_011E', 'B08303_012E', 'B08303_013E', 'NAME']]
            # Renaming columns 
            temp.columns = ['Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
                'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
                'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min', 'State']
            # Adding a column for respective years
            temp['Year'] = file.split(' ')[0]
            # Reordering columns
            temp = temp[['Year', 'State', 'Total Estimate', 'Estimate Less than 5 min', 'Estimate 5 to 9 min', 'Estimate 10 to 14 min', \
                'Estimate 15 to 19 min', 'Estimate 20 to 24 min', 'Estimate 25 to 29 min', 'Estimate 30 to 34 min', 'Estimate 35 to 39 min', \
                'Estimate 40 to 44 min', 'Estimate 45 to 59 min', 'Estimate 60 to 89 min', 'Estimate 90 or more min']]
            # Removing Puerto Rico
            temp = temp[temp['State'].str.contains('Puerto Rico')==False]
            # Appending each new temporary dataframe into one
            travel_df = travel_df.append(temp)
        elif dir == 'Travel Time County and State Level':
            temp = pd.read_csv(path + '\\' + file, skiprows=[1])
            temp = temp[['B08131_001E', 'B08131_002E', 'B08131_003E', 'B08131_004E', 'B08131_005E', 'NAME']]
            # Renaming columns 
            temp.columns = ['Estiamte Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
                'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
                'Worked Outside State of Residence Estimate Aggregate (min)', 'State']
            # Adding a column for respective years
            temp['Year'] = file.split(' ')[0]
            # Reordering columns
            temp = temp[['Year', 'State', 'Estiamte Aggregate Travel Time (min)', 'Worked in State of Residence Estimate Aggregate (min)', \
                'Worked in County of Residence Estimate Aggregate (min)', 'Worked Outside County of Residence Estimate Aggregate (min)', \
                'Worked Outside State of Residence Estimate Aggregate (min)']]
                # Removing Puerto Rico
            temp = temp[temp['State'].str.contains('Puerto Rico')==False]
            # Appending each new temporary dataframe into one
            cs_travel_df = cs_travel_df.append(temp)

# Importing HPI csv and making into a dataframe
hpi_quarterly = pd.read_csv('Resources/HPI/HPI Quarterly State Level.csv')
# Removing Warning column
hpi_quarterly = hpi_quarterly[['state', 'yr', 'qtr', 'index_nsa', 'index_sa']]
# Renaming columns
hpi_quarterly.columns = ['State', 'Year', 'Quarter', 'NSA Index', 'SA Index']
# Gropuby to get mean of SA and NSA index for each state and year
hpi_yearly = hpi_quarterly.groupby(['State', 'Year'], as_index=False).agg({'NSA Index': 'mean', 'SA Index': 'mean'})
# Renaming columns again 
hpi_yearly.columns = ['State', 'Year', 'Yearly Average Index (NSA)', 'Yearly Average Index (SA)']
# Only keeping years 2010-2019
hpi_yearly = hpi_yearly.loc[(hpi_yearly['Year'].astype(int) >= 2010) & (hpi_yearly['Year'].astype(int) <= 2019)]

In [2]:
us_state_to_abbrev = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO", \
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", \
    "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", \
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
"Mississippi": "MS",
"Missouri": "MO",
"Montana": "MT",
"Nebraska": "NE",
"Nevada": "NV",
"New Hampshire": "NH",
"New Jersey": "NJ",
"New Mexico": "NM",
"New York": "NY",
"North Carolina": "NC",
"North Dakota": "ND",
"Ohio": "OH",
"Oklahoma": "OK",
"Oregon": "OR",
"Pennsylvania": "PA",
"Rhode Island": "RI",
"South Carolina": "SC",
"South Dakota": "SD",
"Tennessee": "TN",
"Texas": "TX",
"Utah": "UT",
"Vermont": "VT",
"Virginia": "VA",
"Washington": "WA",
"West Virginia": "WV",
"Wisconsin": "WI",
"Wyoming": "WY",
"District of Columbia": "DC"
}


In [6]:
# Changing state names to abbreviations
travel_df = travel_df.replace({'State': us_state_to_abbrev})
cs_travel_df = cs_travel_df.replace({'State': us_state_to_abbrev})

# Merging HPI and travel time dataframes
travel_hpi = pd.merge(pd.merge(travel_df, cs_travel_df, on=['State', 'Year'], how='left'), hpi_yearly, on=['State', 'Year'], how='left')

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [9]:
travel_df.head()

""


# HPI Summary

# Time Traveled to Work Summary